# LOCALIZACIONES DEFINIDAS
posibles lugares bien situados

In [1]:
london_loc = "51.520624794291884, -0.0870022262169723"

In [2]:
singapore_loc = "1.2862863534171092, 103.84868192263733"

In [3]:
san_francisco_loc = "37.79787988216447, -122.41049370727525"

# API
importar librerias

In [4]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
import geopandas 
import shapely
import pymongo as pym
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry as shg

# FUNCIONES DE LLAMADA A LA API

In [5]:
load_dotenv()
#Tenemos los tokens guardados en un .env en el visual studio code

True

In [6]:
# Asignar todos los token a una variable
tk_id = os.getenv("token_client_id")

In [7]:
tk_secret = os.getenv("token_client_secret")

In [8]:
git_tk = os.getenv("github_tk")

In [9]:
# Asignar una variable a la url
url = 'https://api.foursquare.com/v2/venues/explore'

In [10]:
#pyjsonviewer.view_data(json_data = a)

In [11]:
# A esta función entra una url, unas coordenadas y lo que queremos que nos devuelva la API(query).
# Nos devuelve un lista de diccionarios de sitios con esa descripción cerca de las coordenadas que hemos puesto.
def llamada_API(url, coordenadas, query):
    parametros = {
    "client_id": tk_id,
    "client_secret": tk_secret,
    "v": "20180323",
    "ll": f"{coordenadas}",
    "query": f"{query}",
    "limit": 200  
}
    response = requests.get(url, params=parametros).json()["response"]["groups"][0]["items"]
    return response

In [12]:
# Esta función se mete dentro del la lista de diccionarios previamente creada y nos devuelve todas las variavbles 
# que queremos en una lista de listas. En este caso he escogido el nombre, el tipo de negocio, la latitud, la 
# longitud el país y la dirección 

def escoger_datos(response):
    name = []
    type_ = []
    lat = []
    lng = []
    cc = []
    adress = []
    for i in response:
        name.append(i["venue"]["name"])
    for i in response:
        type_.append(i["venue"]["categories"][0]["name"])
    for i in response:
        lng.append(i["venue"]["location"]["lng"])
    for i in response:
        lat.append(i["venue"]["location"]["lat"])
    for i in response:
        cc.append(i["venue"]["location"]["cc"])
    for i in response:
        adress.append(i["venue"]["location"]["formattedAddress"][0])      
    response2 = [name , type_, cc, adress,  lat, lng ]

    return response2

In [13]:
# Esta función crea un dataframe y lo transpone cambiando las filas por columnas. Además le da un nombre específico
# a cada columna en base a lo que esa columna muestra.
def crear_df(response2):
    df = pd.DataFrame(response2).T
    df1 = df.rename(columns = {0:"Name", 1:"Type", 2:"Country", 3:"Adress", 4:"Latitude", 5:"Longitude", })
    return df1

In [14]:
# Está función junta los valores de latitud y longitud en un punto para que geopandas lo entienda.
def get_geometry(df):
    df2 = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Latitude, df.Longitude))
    return df2

In [15]:
# Esta función le añade cosas a la función anterior para que la localización quede de una manera clara para el 
# ordenador
def get_loc(df):
    df3 = df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return pd.DataFrame(df3)

In [16]:
# Esta función junta todas las funciones anteriores y crea una única función para crear un dataframe claro.
def df_final(url, coordenadas, query):
    a  = llamada_API(url, coordenadas, query)
    b = escoger_datos(a)
    c = crear_df(b)
    d = get_geometry(c)
    d["location"] = get_loc(d)
    df_terminado = d[["Name","Type","Country", "Adress","Latitude","Longitude","location"]]
    return df_terminado

In [17]:
df_final(url, london_loc, "vegan")

,Name,Type,Country,Adress,Latitude,Longitude,location
0,What The Pitta,Vegetarian / Vegan Restaurant,GB,"Unit 52, Boxpark",51.523516,-0.076104,"{'type': 'Point', 'coordinates': (51.523515649..."
1,Vegan Nights London,Vegetarian / Vegan Restaurant,GB,London,51.521603,-0.071401,"{'type': 'Point', 'coordinates': (51.521603, -..."
2,Redemption,Restaurant,GB,320 Old St,51.52663,-0.081919,"{'type': 'Point', 'coordinates': (51.526630326..."
3,Vegan Yes,Vegetarian / Vegan Restaurant,GB,64 Brick Ln,51.518734,-0.071483,"{'type': 'Point', 'coordinates': (51.518734, -..."
4,Genesis,Vegetarian / Vegan Restaurant,GB,144 Commercial St,51.521659,-0.075889,"{'type': 'Point', 'coordinates': (51.521658885..."
...,...,...,...,...,...,...,...
95,Mien Tay,Vietnamese Restaurant,GB,122 Kingsland Rd,51.53044,-0.07718,"{'type': 'Point', 'coordinates': (51.530440436..."
96,The Tea Rooms,Café,GB,Brick Lane (across from alley with Cafe 1001),51.521289,-0.071562,"{'type': 'Point', 'coordinates': (51.521289176..."
97,All Star Lanes,Bowling Alley,GB,95 Brick Ln,51.522026,-0.07183,"{'type': 'Point', 'coordinates': (51.522026307..."
98,Naked Dough,Dessert Shop,GB,Old Street Underground,51.52546,-0.08744,"{'type': 'Point', 'coordinates': (51.525460041..."


# FUNCTIONS TO GET CONDITIONS OF A LOCATION

# SCHOOLS

In [18]:
# Esta función llama a la API para ver todos los colegios que hay cuando le pasas una ubicación, los colegios que
# que busca son los colegios dentro de la lista de colegios. Aparte, añade una columna llamada classification
# al dataframe para saber que son todo colegios.
Schools = ["Nursery School","Elementary School","Middle School","High School"]
def get_schools(location, Schools):
    Nursery_sch = df_final(url, location, Schools[0])
    Elementary_sch = df_final(url, location, Schools[1])
    Middle_sch = df_final(url, location, Schools[2])
    High_sch = df_final(url, location, Schools[3])
    all_schools_bad_index = pd.concat([Nursery_sch, Elementary_sch, Middle_sch, High_sch])
    all_schools = all_schools_bad_index.reset_index(drop=True)
    all_schools.insert(1, "Classification", "School")
    return all_schools

# DESIGNING COMPANIES

In [19]:
#Esta función llama a la API para ver todos los estudios de diseño que hay cuando le pasas una ubicación, aparte 
# tambien mete una columna nueva para aclarar que son estudios de diseño ya que la APi te clasifica lo que te
# devuelve con otras cosas.
def get_design_studio(location):
    DesignCo = df_final(url, location, "Design Studio")
    DesignCo.insert(1, "Classification", "Designing Studios")
    return DesignCo


# TECH STARTUPS

In [20]:
#Esta función llama a la API para ver todos las start ups tecnológicas que hay cuando le pasas una ubicación.
# También añade una columna de clasificación para definirlas como Technological start ups.
def get_tech_startups(location):
    TechStaUp = df_final(url, location, "Tech Startup")
    TechStaUp.insert(1, "Classification", "Technological Startups")
    return TechStaUp

    

# LEISURE

In [21]:
# A esta función le pasas una localización y la palabra leisure. Entonces te devuelve todas las discotecas y sitios
# de ocio cerca de la localización establecida. Además tambien añade una columna clasificando todos esos sitios como
# Leisure.

leisure = ["Nightclub", "Piano Bar", "Pub", "Irish Pub"]
def get_leisure(location, leisure):
    Disco = df_final(url, location, leisure[0])
    Piano_bar = df_final(url, location, leisure[1])
    Pub = df_final(url, location, leisure[2])
    irish_pub = df_final(url, location, leisure[3])
    all_leisure_bad_index = pd.concat([Disco, Piano_bar, Pub, irish_pub])
    leisure = all_leisure_bad_index.reset_index(drop=True)
    leisure.insert(1, "Classification", "Leisure Places")
    return leisure

# TRAVELLING STATIONS

In [22]:
# Esta función recibe una url y la palabra stations. Y te devuelve un dataframe con todas las estaciones de bus,
# de metro y de tren cerca de tu localización. También clasifica todas estas estaciones en una columna, llamda 
# etaciones.
stations = ["Metro Station", "Light Rail Station", "Bus Station"  ]
def get_closeby_stations(location, stations):
    Metro = df_final(url, location, stations[0])
    Rail_Station = df_final(url, location, stations[1])
    Bus_Station = df_final(url, location, stations[1])
    all_stations_bad_index = pd.concat([Metro, Rail_Station, Bus_Station])
    stations = all_stations_bad_index.reset_index(drop=True)
    stations.insert(1, "Classification", "Travelling Stations")
    return stations

# PARKS

In [23]:
# A esta función le pasas una localización y te devuelve todos los parque cercanos. Además tambien incluye una 
# nueva colmna que clasifica a todas las variables como parques
def get_parks(location):
    Parks = df_final(url, location, "Park")
    Parks.insert(1, "Classification", "Parks")
    return Parks

# DATAFRAME GLOBAL Y LIMPIO

In [24]:
# A esta función le pasas una localización y te devuelve la un dataframe global con todo lo previamente calculado 
#con varias columnas.
def df_FINAL(location):
    a = get_schools(location, Schools)
    b = get_design_studio(location)
    c = get_tech_startups(location)
    d = get_leisure(location, leisure)
    e = get_closeby_stations(location,stations)
    f = get_parks(location)
    global_dataframe_bad_index = pd.concat([a,b,c,d,e,f])
    global_df_final = global_dataframe_bad_index.reset_index(drop=True)
    return global_df_final

# LONDON

In [25]:
london_schools = get_schools(london_loc, Schools)
london_designing_companies = get_design_studio(london_loc)
london_tech_startups = get_tech_startups(london_loc)
london_leisure = get_leisure(london_loc, leisure)
london_travelling_stations = get_closeby_stations(london_loc, stations)
london_parks = get_parks(london_loc)
london_df_entero = df_FINAL(london_loc)

In [26]:
# Importar todas las funciones de londres a un csv y el dataframe entero.
london_schools.to_csv("Data/London/ln_schools.csv")
london_designing_companies.to_csv("Data/London/ln_desing.csv")
london_tech_startups.to_csv("Data/London/ln_tech_startups.csv")
london_leisure.to_csv("Data/London/ln_leisure.csv")
london_travelling_stations.to_csv("Data/London/ln_stations.csv")
london_parks.to_csv("Data/London/ln_parks.csv")
london_df_entero.to_csv("Data/London/ln_entero.csv")

In [27]:
london_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
310,Bank London Underground and DLR Station,Travelling Stations,Light Rail Station,GB,Princes St (at Queen Victoria St),51.513376,-0.088931,"{'type': 'Point', 'coordinates': (51.513376461..."
16,New North Primary Academy,School,Elementary School,GB,32 Popham Road,51.538772,-0.095172,"{'type': 'Point', 'coordinates': (51.538771612..."
321,St. Luke's Church Gardens,Parks,Park,GB,161 Old Street,51.524864,-0.093826,"{'type': 'Point', 'coordinates': (51.524863790..."
219,Bricklayer's Arms,Leisure Places,Pub,GB,63 Charlotte Rd. (at Rivington St.),51.526095,-0.081181,"{'type': 'Point', 'coordinates': (51.526095376..."
236,The Anthologist,Leisure Places,Bar,GB,58 Gresham Street,51.514834,-0.091423,"{'type': 'Point', 'coordinates': (51.514834, -..."
275,Ye Olde London,Leisure Places,Pub,GB,42 Ludgate Hill,51.514087,-0.102475,"{'type': 'Point', 'coordinates': (51.514087432..."
47,Camden School for Girls,School,High School,GB,Sandall Street,51.547697,-0.134395,"{'type': 'Point', 'coordinates': (51.547697125..."
234,The Crown Tavern,Leisure Places,Pub,GB,43 Clerkenwell Green (Clerkenwell Cl),51.523034,-0.105243,"{'type': 'Point', 'coordinates': (51.523033614..."
76,Fuerte International,Designing Studios,Design Studio,GB,6-8 Bonhill Street,51.529646,-0.086316,"{'type': 'Point', 'coordinates': (51.529646321..."
50,Idean UK,Designing Studios,Design Studio,GB,"Victoria House, 1 Leonard Circus",51.524766,-0.083781,"{'type': 'Point', 'coordinates': (51.524765823..."


# SINGAPORE

In [28]:
singapore_schools = get_schools(singapore_loc, Schools)
singapore_designing_companies = get_design_studio(singapore_loc)
singapore_tech_startups = get_tech_startups(singapore_loc)
singapore_leisure = get_leisure(singapore_loc, leisure)
singapore_travelling_stations = get_closeby_stations(singapore_loc, stations)
singapore_parks = get_parks(singapore_loc)
singapore_df_entero = df_FINAL(singapore_loc)

In [29]:
singapore_schools.to_csv("Data/Singapore/sg_schools.csv")
singapore_designing_companies.to_csv("Data/Singapore/sg_designco.csv")
singapore_tech_startups.to_csv("Data/Singapore/sg_techstar.csv")
singapore_leisure.to_csv("Data/Singapore/sg_leisure.csv")
singapore_travelling_stations.to_csv("Data/Singapore/sg_stations.csv")
singapore_parks.to_csv("Data/Singapore/sg_parks.csv")
singapore_df_entero.to_csv("Data/Singapore/sg_entero.csv")

In [30]:
singapore_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
213,Drinking Place,Leisure Places,Pub,SG,Singapore,1.286425,103.849821,"{'type': 'Point', 'coordinates': (1.2864246131..."
28,CHIJ St Theresa's Convent,School,High School,SG,160 Lower Delta Rd. (Telok Blangah Way),1.276169,103.822903,"{'type': 'Point', 'coordinates': (1.2761689241..."
40,Queenstown Secondary School,School,High School,SG,1 Strathmore Rd,1.29347,103.813003,"{'type': 'Point', 'coordinates': (1.2934698333..."
292,The Penny Black Public House,Leisure Places,Bar,SG,26/27 Boat Quay (Upper Circular Rd),1.285967,103.850091,"{'type': 'Point', 'coordinates': (1.2859668307..."
354,Bras Basah Park,Parks,Park,SG,9 Bras Basah Rd (opposite the Art Museum),1.298169,103.849187,"{'type': 'Point', 'coordinates': (1.2981691888..."
261,Back Bencher Cafe & Bar,Leisure Places,Pub,SG,Singapore,1.280998,103.845762,"{'type': 'Point', 'coordinates': (1.2809981255..."
87,Asia Square FCC,Technological Startups,Tech Startup,SG,1 Shenton Lane,1.280125,103.851644,"{'type': 'Point', 'coordinates': (1.2801248523..."
285,Duxton Blue,Leisure Places,Pub,SG,35 Duxton Rd,1.277924,103.843112,"{'type': 'Point', 'coordinates': (1.2779235318..."
334,Prudential Tower Garden,Parks,Park,SG,Singapore,1.282738,103.849328,"{'type': 'Point', 'coordinates': (1.2827376802..."
21,QTSS 3E Classroom,School,Middle School,SG,1 Strathmore Road,1.292854,103.812918,"{'type': 'Point', 'coordinates': (1.2928541222..."


# SAN FRANCISCO

In [31]:
sanfrancisco_schools = get_schools(san_francisco_loc, Schools)
sanfrancisco_designing_companies = get_design_studio(san_francisco_loc)
sanfrancisco_tech_startups = get_tech_startups(san_francisco_loc)
sanfrancisco_leisure = get_leisure(san_francisco_loc, leisure)
sanfrancisco_travelling_stations = get_closeby_stations(san_francisco_loc, stations)
sanfrancisco_parks = get_parks(san_francisco_loc)
sanfrancisco_df_entero = df_FINAL(san_francisco_loc)

In [32]:
sanfrancisco_schools.to_csv("Data/SanFrancisco/sf_schools.csv")
sanfrancisco_designing_companies.to_csv("Data/SanFrancisco/sf_design.csv")
sanfrancisco_tech_startups.to_csv("Data/SanFrancisco/sf_techstartups.csv")
sanfrancisco_leisure.to_csv("Data/SanFrancisco/sf_leisure.csv")
sanfrancisco_travelling_stations.to_csv("Data/SanFrancisco/sf_travelling.csv")
sanfrancisco_parks.to_csv("Data/SanFrancisco/sf_parks.csv")
sanfrancisco_df_entero.to_csv("Data/SanFrancisco/sf_entero.csv")

In [33]:
sanfrancisco_df_entero.sample(10)

,Name,Classification,Type,Country,Adress,Latitude,Longitude,location
202,UrbanSitter Home Office,Technological Startups,Tech Startup,US,58 Maiden Ln,37.788527,-122.404569,"{'type': 'Point', 'coordinates': (37.788527, -..."
176,Mobile Crust,Technological Startups,Tech Startup,US,567 Sutter St,37.788692,-122.411041,"{'type': 'Point', 'coordinates': (37.788691856..."
149,Outpace Systems,Technological Startups,Tech Startup,US,475 Sansome St,37.794447,-122.401815,"{'type': 'Point', 'coordinates': (37.794446604..."
86,17FEET,Designing Studios,Design Studio,US,256 Sutter St (Kearny),37.789777,-122.405162,"{'type': 'Point', 'coordinates': (37.789777, -..."
275,Irish Times,Leisure Places,Irish Pub,US,500 Sacramento St (at Sansome St.),37.79401,-122.40169,"{'type': 'Point', 'coordinates': (37.79401, -1..."
206,SFL Data,Technological Startups,Tech Startup,US,100 California St (Davis),37.793977,-122.398,"{'type': 'Point', 'coordinates': (37.793977000..."
58,KIXEYE,Designing Studios,Design Studio,US,333 Bush St (btwn Kearny & Montgomery),37.790862,-122.403126,"{'type': 'Point', 'coordinates': (37.790861523..."
17,Spring Valley Science School,School,Elementary School,US,1451 Jackson St (btwn Hyde & Larkin St),37.794436,-122.418885,"{'type': 'Point', 'coordinates': (37.794436002..."
247,Johnny Foley's Irish House,Leisure Places,Irish Pub,US,243 Ofarrell St (at Cyril Magnin St.),37.78616,-122.408835,"{'type': 'Point', 'coordinates': (37.786159933..."
386,Broadway Tunnel West Mini Park,Parks,Park,US,75 Cyrus (at Broadway),37.796391,-122.417521,"{'type': 'Point', 'coordinates': (37.796391268..."


In [36]:
sanfrancisco_df_entero.shape

(412, 8)

In [38]:
london_df_entero.shape

(355, 8)

In [39]:
singapore_df_entero.shape

(365, 8)